In [ ]:
%load_ext autoreload
%autoreload 2
import time
from continuous_recalibration import ContinuousRecalibration
from read_tags import Detector
import numpy as np
import plotly.express as px

In [ ]:
initial_lparams = np.array([[-5.39622604e-03, -7.55211558e-04, -6.64543613e-03, -1.23351185e-02],
                            [4.89951909e-01, 2.22615412e-01, 2.81593933e-01, 5.03432217e-01],
                            [2.57621490e+00, 4.13689341e+00, 4.23045204e+00, 2.70388820e+00]])
with ContinuousRecalibration(initial_lparams=initial_lparams) as cr:
    cr.update_robot()
    time.sleep(1)
    cr.update_robot()
    t, anchor_locs, ee_loc = cr.read_one_tag()
    print(cr.canonicalize_anchors(anchor_locs))
    print(cr.nearest_robot_ls(t))
    print(cr.calibrate(t, anchor_locs, ee_loc))

In [ ]:
params_history = []
t_history = []
with ContinuousRecalibration(initial_lparams=initial_lparams, lpf_alpha=1) as cr:
    for i in range(100):
        cr.update()
        params_history.append(cr.params.copy())
        t_history.append(cr.last_t)
params_history = np.array(params_history)
t_history = np.array(t_history)

In [ ]:
t = t_history - t_history[0]
fig = px.scatter(x=t, y=params_history[:, 2, 0])
# fig.add_trace(px.scatter(x=t, y=params_history[:, 2, 1]).data[0])
# fig.add_trace(px.scatter(x=t, y=params_history[:, 2, 2]).data[0])
# fig.add_trace(px.scatter(x=t, y=params_history[:, 2, 3]).data[0])
fig.update_traces(mode='lines+markers')

In [ ]:
with ContinuousRecalibration(initial_lparams=initial_lparams, lpf_alpha=0.01) as cr:
    while True:
        cr.update(dry_run=False)